# Redes Neurais
## Prática I - Classificação de padrões

## Base de dados: Ionosphere

In [ ]:
import numpy as np 
import pandas as pd 
import json
import matplotlib.pyplot as plt 
import seaborn as sns 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def piecewise_norm(vec,val,n):
    assert val > min(vec)
    
    return np.where(vec < val, n*(vec - min(vec))/(val - min(vec)), (1-n)*(vec - val)/(max(vec) - val) + n)

In [ ]:
def transform_data(dataset,dataset_columns,normalize=True):
    new_df = pd.DataFrame()
    for name in dataset.columns:
        if name in dataset_columns.keys():
            
            if dataset_columns[name] == 'categorical':
                raw_data = dataset[name].values
                d_encoder = LabelEncoder()
                d_encoder.fit(raw_data)
                d_encoded = d_encoder.transform(raw_data)
                dummy_y = to_categorical(d_encoded)
                #print(dummy_y.shape)
                
                for (j,k) in enumerate(d_encoder.classes_):
                    new_df[f'{name}_{k}'] = dummy_y[:,j].astype('int')
            
            
            elif dataset_columns[name] == 'numeric':
                raw_data = dataset[name].values
                if normalize:
                    new_df[name] = (raw_data - min(raw_data))/(max(raw_data) - min(raw_data))
                else:
                    new_df[name] = raw_data
                
            elif dataset_columns[name] == 'original':
                new_df[name] = dataset[name].values
            
            elif dataset_columns[name] == 'target':
                new_df[name] = dataset[name].values
                
    return new_df

In [ ]:
def create_model(hidden_neurons = [4], hidden_activation = ['relu'], output_activation='softmax', lr = 0.05, n_input = 1, n_output = 1):
    # create model
    model = Sequential()
    model.add(Dense(hidden_neurons[0], input_dim=n_input, activation='relu'))
    for i in range(1,len(hidden_neurons)):
        model.add(Dense(hidden_neurons[i], input_dim=hidden_neurons[i-1], activation='relu'))
        

    model.add(Dense(n_output, activation=output_activation))
    # Compile model
    opt = Adam(lr=lr)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Preparação dos dados

In [ ]:
#with open('data_info.json','r') as f:
#    dataset_columns = json.load(f)

In [ ]:
dataset = pd.read_csv('ionosphere.data')

## Parte I - Compreensão do problema e análise de variáveis



### 1)	Observe a base de dados do problema. Existem variáveis que podem ser eliminadas do dataset? Justifique.

#### Rascunho

#### Resposta

In [ ]:
dataset.head()

In [ ]:
dataset.drop(labels=['ID'],axis=1,inplace=True)

### 2)	Implemente técnicas de visualização de dados e seleção de variáveis para extrair características importantes sobre a base de dados. Explique a motivação destas técnicas e o que é possível inferir dos resultados obtidos.

#### Rascunho

#### Resposta

In [ ]:
#sns.pairplot(dataset[['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compacity','concavity', 'pts_concavity', 'symmetry', 'fractal','target']], hue="target", diag_kind="hist")
sns.pairplot(dataset[['radius', 'texture', 'perimeter', 'area', 'target']], hue="target", diag_kind="hist")

## 2.	Treinamento do modelo de Rede Neural

### 1)	Com as configurações do modelo MLP previamente definidas no script, faça o treinamento da Rede Neural sem normalizar os atributos numéricos. Comente o resultado obtido, baseado nas métricas de avaliação disponíveis (acurácia, precision, recall, F1-Score, Matriz de confusão, etc.)

#### Rascunho

In [ ]:
#Parâmetros do modelo 

hidden_neurons = [15]
activation_layers = ['relu']

output_activation = 'sigmoid'

lr = 0.05
epochs = 50

In [ ]:
new_df = transform_data(dataset,dataset_columns,normalize=False)
new_df.head()

In [ ]:
X = new_df.values[:,:-1].astype('float')
target = new_df.values[:,-1]

encoder = LabelEncoder()
Y = encoder.fit_transform(target)

n_input = X.shape[1]
n_output = 1

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
model = create_model(hidden_neurons=hidden_neurons,output_activation=output_activation,n_input=n_input,n_output=n_output, lr = lr)

In [ ]:
model.summary()

#### Resposta

In [ ]:
model.fit(x=x_train,y=y_train,epochs=epochs)

In [ ]:
raw_Y_pred = model.predict(x_test)
y_pred = np.where(raw_Y_pred > 0.5, 1, 0)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print('Classification Report')
target_names = ['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:

cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cmn,annot=True)
plt.ylabel('Actual')
plt.xlabel('Predicted')

### 2)	Agora normalize os dados de entrada e treine novamente o modelo MLP. Avalie os resultados obtidos e comente o efeito da normalização no treinamento da Rede Neural.

#### Rascunho

In [ ]:
new_df = transform_data(dataset,dataset_columns,normalize=True)
new_df.head()

In [ ]:
del model 


#### Resposta

In [ ]:
model.fit(x=x_train,y=y_train,epochs=epochs)


## Parte III - Mudança de configurações do modelo

### 1)	Insira o conjunto de validação para o treinamento do modelo. Avalie o resultado obtido.

#### Rascunho

In [ ]:
del model


#### Resposta

### 2)	Modifique o tempo de treinamento (épocas) da Rede Neural. Escolha dois valores distintos (e.g. 1 e 1000 épocas) e avalie os resultados.

#### Rascunho

#### Resposta

### 3)	 Modifique a taxa de aprendizado da Rede Neural. Escolha dois valores distintos (e.g. 0,001 e 0,1) e avalie os resultados.

#### Rascunho

#### Resposta

### 4)	Modifique a quantidade de neurônios na camada escondida da Rede Neural. Escolha dois valores distintos (e.g. 2 e 70 neurônios) e avalie os resultados.

#### Rascunho

#### Resposta

## Parte IV - Análise dos resultados

### 1)	Faça novos testes para avaliar o desempenho da Rede Neural no problema designado. Use a técnica K-Fold (com K = 10) para analisar o resultado obtido.

#### Rascunho

#### Resposta

### 2)	Faça análises e novas implementações que você julgue importante para o seu trabalho. Não esqueça de explicar a motivação da análise realizada. 

#### Rascunho

#### Resposta